# 全方位静态类型系统

## 4.1 类型提示：提高代码表达力

### 4.1.1 泛型：处理容器与抽象类型

泛型允许我们创建可重用的组件，这些组件能够处理多种类型而不会牺牲类型安全性。

#### 基本容器类型注解

Python 提供了标准库中的泛型容器类型，用于声明容器内元素的类型：

In [1]:
from typing import List, Dict, Tuple


# 基本容器类型注解
def process_users(users: List[str]) -> Dict[str, int]:
    """
    处理用户列表，返回用户名称和长度的字典。
    """
    return {user: len(user) for user in users}


# Python 3.9+ 可以使用原生语法
def process_data(data: list[str]) -> dict[str, int]:
    return {item: len(item) for item in data}


users = ["张三", "李四", "王小五", "赵老六"]

result = process_users(users)
print(result)  # 输出: {'张三': 2, '李四': 2, '王小五': 3, '赵老六': 3}

{'张三': 2, '李四': 2, '王小五': 3, '赵老六': 3}


#### 自定义泛型类

In [ ]:
from typing import TypeVar

# 1. 无约束的 TypeVar：可以代表任何类型
T = TypeVar("T")  # 'T' 是类型变量的名称（惯例用单个大写字母）

# 2. 有约束的 TypeVar：只能代表指定类型（比如 int 或 str）
S = TypeVar("S", int, str)  # S 只能是 int 或 str

# 3. 绑定 TypeVar：指定类型变量的上界（即只能是指定类型或其子类）
V = TypeVar("V", bound=int)  # V 只能是 int 或其子类(python中，bool是int的子类)


def first(values: list[V]) -> V:
    return values[0]


# Python 解释器本身不强制执行类型注解，所以这一步不会直接导致程序崩溃,
# 但不符合类型设计的意图。如果用静态检查工具（如 mypy）运行，也会直接提示错误：
print(first(["hello", "world"]))

hello


In [ ]:
from typing import TypeVar, Generic, Callable
from datetime import date

# 类型变量：T 表示“原始类型”，U 表示“转换后的类型”
T = TypeVar("T")
U = TypeVar("U")


class Container(Generic[T]):
    """
    一个自定义的泛型容器，可以安全地保存任意类型的值，
    并通过 transform 方法对值进行类型安全的转换。
    """

    def __init__(self, value: T) -> None:
        # 保存传入的值，类型为 T
        self.value = value

    def get_value(self) -> T:
        # 返回内部保存的值，类型仍然是 T
        return self.value

    def transform(self, func: Callable[[T], U]) -> "Container[U]":
        """
        接收一个函数 func: T -> U，
        把当前容器里的值从 T 类型转换成 U 类型，
        并返回一个新的 Container[U]，保证类型信息不丢失。
        """
        transformed_value = func(self.value)  # 执行转换
        return Container(transformed_value)  # 新容器，类型为 Container[U]


# 使用示例


# 1. 定义一个转换函数：把 date 格式化成中文年月日
def format_chinese(d: date) -> str:
    """
    输入: date(2035, 11, 19)
    输出: '2035年11月19日'
    """
    return f"{d.year}年{d.month}月{d.day}日"


# 2. 创建一个装 date 的容器
date_container: Container[date] = Container(date(2035, 11, 19))

# 3. 使用 transform 应用上面的格式化函数
str_container: Container[str] = date_container.transform(format_chinese)

# 4. 取出结果，已经是 str 类型
print(str_container.get_value())  # 输出: 2035年11月19日

2035年11月19日


#### 协变逆变

In [10]:
from typing import TypeVar, Generic


class Fruit:
    def __str__(self):
        return "fruit"


class Apple(Fruit):
    def __str__(self):
        return "apple"


# 定义类型变量
T = TypeVar("T")


# 定义一个泛型的切水果工具
class Cutter(Generic[T]):
    def process(self, item: T) -> None:
        pass


# 切水果工具：可以处理任何 Fruit
class FruitCutter(Cutter[Fruit]):
    def cut(self, fruit: Fruit) -> None:
        print(f">>>>通用切水果工具正在切: {fruit}")


# 切苹果工具：专门处理 Apple
class AppleCutter(Cutter[Apple]):
    def cut(self, apple: Apple) -> None:
        print(f">>>>专用切苹果工具正在切: {apple}")


# --- 核心：逆变的体现 ---
# 假设我们有一个函数，它需要一个切苹果的工具作为参数
def cut_apple(cutter: Cutter[Apple], apple: Apple) -> None:
    print(">>>>准备切苹果...")
    cutter.cut(apple)


# 1. 使用专门的 AppleCutter（这是最直接的用法）
apple_cutter = AppleCutter()
apple = Apple()
cut_apple(apple_cutter, apple)
# 输出:
# >>>>准备切苹果...
# >>>>专用切苹果工具正在切: apple

# 2. 使用通用的 FruitCutter（这就是逆变！）
# 虽然 cut_apple 函数声明需要 Cutter[Apple]，
# 但我们传入了一个 Processor[Fruit]，这在类型上是安全的。
fruit_cutter = FruitCutter()
cut_apple(fruit_cutter, apple)
# 输出:
# >>>>准备切苹果...
# >>>>通用切水果工具正在切: apple

>>>>准备切苹果...
>>>>专用切苹果工具正在切: apple
>>>>准备切苹果...
>>>>通用切水果工具正在切: apple


### 4.1.2 协议：实现鸭子类型的类型安全

In [17]:
from typing import Protocol, runtime_checkable
from dataclasses import dataclass


# 定义可食用水果的协议
class EdibleFruit(Protocol):
    name: str
    color: str

    def eat(self) -> str:
        """食用水果的方法"""
        ...

    @property
    def calories(self) -> float:
        """计算热量"""
        ...


# 定义可榨汁水果的协议
class Juiceable(Protocol):
    def make_juice(self, quantity: int) -> str:
        """榨汁方法"""
        ...


# 创建组合协议
@runtime_checkable
class EdibleAndJuiceable(EdibleFruit, Juiceable, Protocol):
    """同时可食用和可榨汁的水果协议"""

    pass


# 使用协议进行类型注解
def prepare_fruit_snack(fruit: EdibleAndJuiceable) -> tuple[str, str]:
    """准备水果零食：食用并榨汁"""
    eat_result = fruit.eat()
    juice_result = fruit.make_juice(250)
    return eat_result, juice_result


# 实现协议的类（无需显式继承）
@dataclass
class Apple:
    name: str = "苹果"
    color: str = "红色"
    variety: str = "富士"

    def eat(self) -> str:
        return f"吃了一个{self.color}的{self.variety}{self.name}"

    @property
    def calories(self) -> float:
        return 52.0

    def make_juice(self, quantity: int) -> str:
        return f"榨了{quantity}ml的{self.variety}{self.name}汁"


@dataclass
class Orange:
    name: str = "橙子"
    color: str = "橙色"
    is_sweet: bool = True

    def eat(self) -> str:
        sweetness = "甜" if self.is_sweet else "酸"
        return f"剥开了一个{self.color}的{sweetness}{self.name}"

    @property
    def calories(self) -> float:
        return 47.0

    def make_juice(self, quantity: int) -> str:
        sweetness = "甜" if self.is_sweet else "酸"
        return f"榨了{quantity}ml的{sweetness}{self.name}汁"


# 类型检查通过
apple = Apple()
orange = Orange()

# 任何实现了协议方法的类都可以使用
snack1 = prepare_fruit_snack(apple)
snack2 = prepare_fruit_snack(orange)

print(snack1)  # ('吃了一个红色的富士苹果', '榨了250ml的富士苹果汁')
print(snack2)  # ('剥开了一个橙色的甜橙子', '榨了250ml的甜橙子汁')

# 运行时类型检查，虽然Apple/Orange没有在代码中直接继承EdibleAndJuiceable协议
# 但它们实现了协议的所有方法，因此它们符合协议的定义，下面语句返回True
print(f"苹果符合协议: {isinstance(apple, EdibleAndJuiceable)}")  # True
print(f"橙子符合协议: {isinstance(orange, EdibleAndJuiceable)}")  # True


# 不符合协议的类会在类型检查时报错
class Rock:
    """石头类 - 不符合水果协议"""

    name = "石头"
    color = "灰色"

    def eat(self) -> str:
        return "尝试吃石头... 牙齿坏了！"

    # 缺少 calories 属性和 make_juice 方法


# 下面的代码在静态类型检查时会报错：
# rock = Rock()
# prepare_fruit_snack(rock)  # 错误：Rock 不符合 EdibleAndJuiceable 协议

('吃了一个红色的富士苹果', '榨了250ml的富士苹果汁')
('剥开了一个橙色的甜橙子', '榨了250ml的甜橙子汁')
苹果符合协议: True
橙子符合协议: True


#### 4.1.3 类型别名与 NewType：简化复杂声明

#### 类型别名简化复杂签名

In [19]:
from typing import Dict, List, Union, TypeAlias

# 复杂的类型签名
UserData = Dict[str, Union[int, str, List[str]]]


# 使用类型别名
def process_user_data(data: UserData) -> None:
    name = data.get("name", "")
    scores = data.get("scores", [])
    print(f"User {name}: {scores}")


# Python 3.10+ 可以使用更简洁的语法
ResponseData: TypeAlias = dict[str, list[int] | str | None]


def handle_response(response: ResponseData) -> bool:
    return response.get("status") == "success"

#### NewType

In [22]:
from typing import NewType

# 创建语义不同的类型
UserId = NewType("UserId", int)
PostId = NewType("PostId", int)
Email = NewType("Email", str)


def get_user_profile(user_id: UserId) -> dict:
    return {"user_id": user_id, "name": "小白"}


def get_post_content(post_id: PostId) -> str:
    return f"Content of post {post_id}"


# 使用时需要显式转换
raw_id = 123
user_id = UserId(raw_id)
post_id = PostId(raw_id)

# 类型检查通过
user_data = get_user_profile(user_id)
post_content = get_post_content(post_id)

# 类型错误！期望 UserId，传入 PostId
user_data_error = get_user_profile(post_id)  # 代码能运行，但类型检查器会报错
print(user_data_error)

{'user_id': 123, 'name': '小白'}


## 4.2 应对动态特性：Any 的合理使用与类型忽略

### 4.1 合理使用任意类型Any

In [ ]:
from typing import Any, cast
import json


# ======== 场景1：处理不确定的外部数据 ========
def parse_json_data(json_str: str) -> Any:
    """从外部 API 解析 JSON 数据，类型无法提前确定"""
    return json.loads(json_str)


def process_external_data() -> dict[str, int]:
    # 从外部源获取数据
    raw_data = parse_json_data('{"count": 100, "value": 42}')

    # 不要直接采用parse_json_data()返回Any类型，而是立即转换回具体类型，缩小影响范围
    if isinstance(raw_data, dict):
        return cast(dict[str, int], raw_data)
    else:
        return {}


# ======== 场景2：对比好的 vs 坏的设计 ========


# 错误示范：将 Any 暴露在公共 API 中，让调用方猜谜
def bad_api() -> Any:  # 避免在公共接口中使用 Any！
    return {"data": "anything"}


# 正确做法：明确返回具体类型
def good_api() -> dict[str, str | int]:
    """明确告诉调用方：返回字典，键是字符串，值是字符串或整数"""
    return {"name": "小白", "age": 28, "city": "北京"}

## 4.2.2 # type: ignore

下面的代码运行正常，但不能通过mypy类型检查

In [27]:
from typing import List

# 创建一个整数列表的类型注解
items: List[int] = [1, 2, 3]

# 这行代码会导致类型检查错误：尝试向整数列表添加字符串
items.append("string")  # 这里会引发类型错误

# 使用 type: ignore 来抑制类型检查错误
items.append("string")  # type: ignore

# 转换列表中的所有元素为字符串
string_items = [str(item) for item in items]

print(f"字符串列表: {string_items}")

字符串列表: ['1', '2', '3', 'string', 'string']


## 4.4 基于Pydantic的运行时类型校验

In [ ]:
from pydantic import BaseModel, ValidationError
from typing import List, Optional
from datetime import datetime


class User(BaseModel):
    id: int
    name: str
    email: str
    signup_date: datetime
    tags: List[str] = []
    age: Optional[int] = None


# 自动验证和转换示例
try:
    user_data = {
        "id": "1",  # 字符串会被自动转换为整数
        "name": "小白",
        "address": "星辰大海",  # 多余字段会被忽略
        "email": "xiaobai@example.com",
        "signup_date": "2035-01-15",
        "tags": ["python", "developer"],
        "age": "25",
    }
    user = User(**user_data)
    print("id: ", user.id)  # 输出: 1 (整数)
    print("signup date: ", user.signup_date)  # 输出: datetime对象
except ValidationError as e:
    print(f"验证错误: {e}")

id:  1
signup date:  2035-01-15 00:00:00


In [ ]:
from pydantic import BaseModel, field_validator
from typing import List


class Product(BaseModel):
    name: str
    price: float
    quantity: int
    categories: List[str]

    @field_validator("name")
    def name_must_not_empyt(cls, v) -> str:
        """验证产品名称必须包含内容"""
        if len(v) == 0:
            raise ValueError("产品名称必须包含内容")
        return v.title()  # 自动转换为标题格式

    @field_validator("price")
    def price_must_be_positive(cls, v) -> float:
        """验证价格必须为正数"""
        if v <= 0:
            raise ValueError("价格必须为正数")
        return v

    @field_validator("categories")
    def categories_non_empty(cls, v) -> list[str]:
        """验证至少需要一个分类"""
        if not v:
            raise ValueError("至少需要一个产品分类")
        return v


# 序列化和反序列化操作
product_data = {
    "name": "笔记本电脑",
    "price": 5999.99,
    "quantity": 10,
    "categories": ["电子产品", "笔记本"],
}

product = Product(**product_data)
print(product.model_dump())  # 序列化为字典
print(product.model_dump_json())  # 序列化为JSON字符串

{'name': '笔记本电脑', 'price': 5999.99, 'quantity': 10, 'categories': ['电子产品', '笔记本']}
{"name":"笔记本电脑","price":5999.99,"quantity":10,"categories":["电子产品","笔记本"]}
